# Wind Farm Spotter: Inference Test (Single) [Swift, CPU Runtime]

### **Environment Setup**
First setting up the colab/jupyter notebook with necessary Swift packages.

In [0]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JohnSundell/ShellOut.git", from: "2.0.0")' ShellOut
%install '.package(url: "https://github.com/IBM-Swift/SwiftyRequest.git", from: "2.0.6")' SwiftyRequest
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just
%install '.package(url: "https://github.com/JohnSundell/Files.git", from: "3.1.0")' Files
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport
%install '.package(url: "https://github.com/weichsel/ZIPFoundation/", .upToNextMajor(from: "0.9.0"))' ZIPFoundation
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

**Swift Imports:**

In [0]:
import Foundation
import Python
import SwiftyRequest
import Path
import Files
import ShellOut
import FoundationNetworking
import ZIPFoundation

**Python Imports:**

We'll be using the shellOut package to run a shell command to **Install pytorchcv** and have access to the necessary PyTorch base architecture implementations.

In [3]:
try shellOut(to: "pip install pytorchcv")

"Requirement already satisfied: pytorchcv in /usr/local/lib/python3.6/dist-packages (0.0.52)\nRequirement already satisfied: requests in /usr/local/lib/python3.6/dist-packages (from pytorchcv) (2.21.0)\nRequirement already satisfied: numpy in /usr/local/lib/python3.6/dist-packages (from pytorchcv) (1.17.4)\nRequirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (3.0.4)\nRequirement already satisfied: urllib3<1.25,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (1.24.3)\nRequirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (2019.9.11)\nRequirement already satisfied: idna<2.9,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (2.8)"


In [0]:
let warnings = Python.import("warnings")
warnings.filterwarnings("ignore")
let os = Python.import("os")
let pd = Python.import("pandas")
let fastai = Python.import("fastai")
let fastai_vision = Python.import("fastai.vision") 
let plt = Python.import("matplotlib.pyplot")
let py_time = Python.import("time")
let sys = Python.import("sys")
let torch = Python.import("torch")
let pytorchcv = Python.import("pytorchcv")
let model_provider = Python.import("pytorchcv.model_provider")


In [5]:
print("Fastai Version:", fastai.__version__)

Fastai Version: 1.0.59


**CUDA Check:**

In [6]:
torch.cuda.is_available()

False


### **Accessing the Image Data**

In [7]:
print(Folder.current)

Folder(name: content, path: /content/)


In [0]:
let fileManager = FileManager()

In [0]:
let test_data_zip = Path("/content/test_single.zip")!
fileManager.unzipItem(at: test_data_zip.url, to: Path("/content/")!.url)

### **Reloading the Fast.ai Learner**
Fast.ai provides a useful method named **load_learner()** to reinstantiate an instance model from a pickle file. The loaded learner recalls all the configurations(e.g., transformations, batch size, etc.) used during training. load_learner() also gives us the option of assigning a list of images (unlabeled) right from the start. To run our Single Image Inference Test, we'll use the single_test directory to create a Fast.ai **ImageList**.

In [0]:
let ImageList = fastai_vision.ImageList
//let ptcv_get_model = model_provider.get_model 

In [0]:
let inference_test_efficientnet = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"effnet-b1-c98.pkl", "test": ImageList.from_folder("/content/test_single")])
let inference_test_inceptionv3 = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"incept-v3-95-2.pkl", "test": ImageList.from_folder("/content/test_single")])

**Create a name list:**

In [0]:
let test_files = Path("/content/test_single")!.url.path
let test_file_list = os.listdir(test_files)
let df = pd.DataFrame(test_file_list)
//df

**EfficientNet Single Inference:**

Here, we set our model to evaluation mode...

In [0]:
inference_test_efficientnet.model.eval()

and loop through the learner's test dataset, making a prediction for each item, and storing the result in our eff_preds dictionary: 

In [14]:
var eff_preds:[String:String] = [:]

let eff_start = py_time.time()
for img in 0...48 {
  var name = String(test_file_list[img])!
  var p = inference_test_efficientnet.predict(inference_test_efficientnet.data.test_dl.dl.dataset.x[img]).tuple3
  eff_preds[name] = String(Python.str(p.0))!
}
let eff_stop = py_time.time()
let eff_elapsed = eff_stop - eff_start
print("Swift Elapsed Time: \(eff_elapsed)")


Swift Elapsed Time: 34.425660610198975


In [0]:
let eff_pred_list:PythonObject = PythonObject(eff_preds).items()
//print(pred_list)

**Results:**

In [16]:
let eff_results = pd.DataFrame.dynamicallyCall(withKeywordArguments: ["data": Array(eff_pred_list), "columns": ["file_name", "id"]])
print(eff_results)

                                file_name                          id
0   test_no_turbines_high_potential_2.jpg  no_turbines_high_potential
1    test_no_turbines_med_potential_3.jpg   no_turbines_med_potential
2   test_no_turbines_high_potential_5.jpg  no_turbines_high_potential
3                test_high_capacity_5.jpg      turbines_high_capacity
4    test_no_turbines_med_potential_2.jpg   no_turbines_med_potential
5                test_high_capacity_8.jpg      turbines_high_capacity
6                test_high_capacity_2.jpg      turbines_high_capacity
7              test_medium_capacity_4.jpg   no_turbines_low_potential
8                 test_low_capacity_5.jpg   no_turbines_low_potential
9              test_medium_capacity_6.jpg    turbines_medium_capacity
10            test_medium_capacity_13.jpg    turbines_medium_capacity
11   test_no_turbines_low_potential_1.jpg   no_turbines_low_potential
12             test_medium_capacity_7.jpg   no_turbines_med_potential
13                te

**Inception-v3 Single Inference:**

We'll do the same for Inception-v3:

In [0]:
inference_test_inceptionv3.model.eval()

In [18]:
var incept_preds:[String:String] = [:]

let incept_start = py_time.time()
for img in 0...48 {
  var name = String(test_file_list[img])!
  var p = inference_test_inceptionv3.predict(inference_test_inceptionv3.data.test_dl.dl.dataset.x[img]).tuple3
  incept_preds[name] = String(Python.str(p.0))!
}

let incept_stop = py_time.time()
let incept_elapsed = incept_stop - incept_start 
print("Elapsed Time: \(incept_elapsed)")

Elapsed Time: 34.97622561454773


In [0]:
let incept_pred_list:PythonObject = PythonObject(incept_preds).items()
//print(pred_list)

**Results:**

In [20]:
let incept_results = pd.DataFrame.dynamicallyCall(withKeywordArguments: ["data": Array(incept_pred_list), "columns": ["file_name", "id"]])
print(incept_results)

                                file_name                          id
0   test_no_turbines_high_potential_2.jpg  no_turbines_high_potential
1             test_medium_capacity_10.jpg    turbines_medium_capacity
2    test_no_turbines_med_potential_3.jpg   no_turbines_med_potential
3                test_high_capacity_2.jpg    turbines_medium_capacity
4    test_no_turbines_med_potential_1.jpg   no_turbines_med_potential
5     test_no_turbines_no_potential_6.jpg    no_turbines_no_potential
6                 test_low_capacity_3.jpg       turbines_low_capacity
7              test_medium_capacity_6.jpg    turbines_medium_capacity
8    test_no_turbines_low_potential_1.jpg   no_turbines_low_potential
9                test_high_capacity_7.jpg      turbines_high_capacity
10                test_low_capacity_2.jpg       turbines_low_capacity
11               test_low_capacity_10.jpg       turbines_low_capacity
12                test_low_capacity_5.jpg   no_turbines_low_potential
13    test_no_turbin